In [31]:
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat 

In [32]:
B = 512;C = 10000;W = 1.4 * pow(10,6)
w_I = w_M = pow(10,-30)
f_I = pow(10,7);f_M = pow(10,12)
N_0 = -132.24;P = 0.5 

def arrival_and_departure_times(param1,param2,N):
    a = []
    d = []
    s = 0
    for i in range(N):
        s = s + np.random.exponential(param1)
        a.append(s)
    for i in range(N):
        d.append(a[i] + np.random.exponential(param2))
    return (np.array(a),np.array(d))

def local_cost():
    return ((B*C)/f_I)

def cloud_cost(n):
    h_g_2 = np.random.exponential(1/0.19)
    R = W * np.log2(1 + P*h_g_2/N_0)
    return (B/R + B*C*n/f_M)

def next_state(local_pos,cloud_pos,action):
    if action == 0:
        (new_local_pos,new_cloud_pos,reward) = (local_pos + 1,cloud_pos,-local_cost())
    else:
        (new_local_pos,new_cloud_pos,reward) = (local_pos,cloud_pos + 1,-cloud_cost(cloud_pos));
    return  (new_local_pos,new_cloud_pos,reward);


In [37]:

def Server_Q_learning(N,gamma):
    limit = 10
    reward_list = []
    reward_sum = 0
    count = 0
    Qa = np.zeros((10,10,2))
    freq = np.ones((10,10))
    freq_action = np.ones((10,10,2))
    local_actions = np.array([False]*limit)
    cloud_actions = np.array([False]*limit)
    mu = 1;lamda = 1
    (arrivals,departures) = arrival_and_departure_times(mu,lamda,N)
    local_pos = 0;cloud_pos = 0
    for i in range(N):
        local_pos = ((departures > arrivals[i]) & (arrivals < arrivals[i]) & (local_actions)).sum()
        cloud_pos = ((departures > arrivals[i]) & (arrivals < arrivals[i]) & (cloud_actions)).sum()
        eps = 1/np.sqrt(freq[local_pos,cloud_pos])
        action = np.random.choice([np.argmax(Qa[local_pos,cloud_pos]),np.random.choice([0,1])],p = [1 - eps,eps])
        if local_pos == limit:
            action = 1
        if cloud_pos == limit:
            action = 0        
        if action == 0:
            local_actions[i] = 1
        if action == 1:
            cloud_actions[i] = 1
        (new_local_pos,new_cloud_pos,reward) = next_state(local_pos,cloud_pos,action)
        reward_sum = reward_sum + reward
        count = count + 1
        alpha = 1/np.power(freq_action[local_pos,cloud_pos, action],0.8)
        Qa[local_pos,cloud_pos,action] = Qa[local_pos,cloud_pos,action] + alpha*(reward + gamma*np.max(Qa[new_local_pos,new_cloud_pos]) - Qa[local_pos,cloud_pos,action])
        freq[local_pos,cloud_pos] = freq[local_pos,cloud_pos] + 1
        freq_action[local_pos,cloud_pos,action] = freq_action[local_pos,cloud_pos,action] + 1     
        reward_list.append(reward_sum/count)
    return reward_list

In [40]:
Server_Q_learning(100,0.05)

0
1
2
3
4
5
6
7
8
9


[-0.512,
 -0.24895148497874708,
 -0.161942688243276,
 -0.11507820536760138,
 -0.08977460405095408,
 -0.07209717679410696,
 -0.058819273148088824,
 -0.04883654310037944,
 -0.1002991494225595,
 -0.08967449611398079]

numpy.ndarray

20